[Solvers, Optimizers, and Automatic Differentiation]("https://julia.quantecon.org/more_julia/optimization_solver_packages.html")

In [1]:
using InstantiateFromURL
# optionally add arguments to force installation: instantiate = true, precompile = true
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.8.0")

Activated /Users/taisei/Project.toml
Info Project name is quantecon-notebooks-julia, version is 0.8.0


In [2]:
using LinearAlgebra, Statistics
using ForwardDiff, Optim, JuMP, Ipopt, BlackBoxOptim, Roots, NLsolve, LeastSquaresOptim

# 

In [3]:
#===using Pkg
Pkg.rm("Zygote")
==#

   Updating `~/Project.toml`
  [e88e6eb3] - Zygote v0.3.2
   Updating `~/Manifest.toml`
  [7869d1d1] - IRTools v0.2.3
  [e88e6eb3] - Zygote v0.3.2


In [5]:
#using Zygote


┌ Info: Precompiling Zygote [e88e6eb3-aa80-5325-afca-941959d7151f]
└ @ Base loading.jl:1260


# Introduction to Differentiable Programming

Three ways to calculate the gradient or Jacobian

1. Analytic derivatives/Symbolic differentiation

2. Finite differences
Tradeoff: Large $\Delta$ is numerically stable but inaccurate.
Use `DiffEqDiffTools.jl` to choose $\Delta$

3. Automatic Differentiation
The same as analytic/symbolic differentiation, but where the **chain rule** is calculated **numerically**.
e.g. d(sin(x))=>cos(x)dx

Two basic approaces of AD

1. reverse-mode : when a function is R^n to R

2. forward-mode: when a function is R to R^N

* chain rule

$\frac{dy}{dx}=\frac{dy}{dw}\frac{dw}{dx}$

Forward-mode starts the calculation from the left with $\frac{dy}{dw}$ then  calculates the product with $\frac{dw}{dx}$.

Reverse mode starts on the right hand side with $\frac{dw}{dx}$ and works backwards

## Example

$$f(x_1,x_2)=x_1x_2+\sin(x_1)$$

In [6]:
function f(x_1, x_2)
    w_1 = x_1
    w_2 = x_2
    w_3 = w_1 * w_2
    w_4 = sin(w_1)
    w_5 = w_3 + w_4
    return w_5
end

f (generic function with 1 method)

Note: The derivative of w_1 with respect to x_1 and w_2 with respect to x_2 are dual numbers.
So if we have the function $f(x_1,x_2)$ and want to find the derivative of f(3.8,6.9), we would see them with the dual numbers x_1=(3.8, 1) and x_2= (6.9,0)

### Forward-Mode Automatic Differentiation

First fix the variable you are interested in (called **seeding**)

In [7]:
using ForwardDiff
h(x) = sin(x[1]) + x[1] * x[2] + sinh(x[1] * x[2]) # multivariate.
x = [1.4 2.2]
@show ForwardDiff.gradient(h,x) # use AD, seeds from x

#Or, can use complicated functions of many variables
f(x) = sum(sin, x) + prod(tan, x) * sum(sqrt, x)
g = (x) -> ForwardDiff.gradient(f, x); # g() is now the gradient
g(rand(5)) # gradient at a random point
# ForwardDiff.hessian(f,x') # or the hessian

ForwardDiff.gradient(h, x) = [26.354764961030977 16.663053156992284]


5-element Array{Float64,1}:
 0.9172415083404515
 1.3255149730878326
 1.1401222603500552
 1.156746730742981
 1.0846333355946316

In [11]:
function squareroot(x) #pretending we don't know sqrt()
    z = copy(x) # Initial starting point for Newton’s method
    while abs(z*z - x) > 1e-13
        z = z - (z*z-x)/(2z)
    end
    return z
end
squareroot(2.0)

1.4142135623730951

### Reverse-mode : `Zygote.jl`

In [8]:
using Zygote

h(x, y) = 3x^2 + 2x + 1 + y*x - y
gradient(h, 3.0, 5.0)

(25.0, 2.0)

In [9]:
using Optim, LinearAlgebra
N = 1000000
y = rand(N)
λ = 0.01
obj(x) = sum((x .- y).^2) + λ*norm(x)

x_iv = rand(N)
function g!(G, x)
    G .=  obj'(x)
end

results = optimize(obj, g!, x_iv, LBFGS()) # or ConjugateGradient()
println("minimum = $(results.minimum) with in "*
"$(results.iterations) iterations")

minimum = 5.774181450804439 with in 2 iterations


In [10]:
D(f) = x-> gradient(f, x)[1]  # returns first in tuple

D_sin = D(sin)
D_sin(4.0)

-0.6536436208636119

# Optimization

### Univariate Functions on Bounded Intervals

In [2]:
using Optim
using Optim: converged, maximum, minimizer, iterations

result = optimize(x->x^2,-2.0,1.0)

Results of Optimization Algorithm
 * Algorithm: Brent's Method
 * Search Interval: [-2.000000, 1.000000]
 * Minimizer: 0.000000e+00
 * Minimum: 0.000000e+00
 * Iterations: 5
 * Convergence: max(|x - x_upper|, |x - x_lower|) <= 2*(1.5e-08*|x|+2.2e-16): true
 * Objective Function Calls: 6

In [2]:
converged(result)||error("Failed to converge in $(iterations(result)) iterations")
xmin = result.minimizer
result.minimum

0.0

In [5]:
using Optim: maximizer
f(x) = -x^2
result = maximize(f, -2.0, 1.0)
converged(result) || error("Failed to converge in $(iterations(result)) iterations")
xmin = maximizer(result)
fmax = maximum(result)

-0.0

### Unconstrained Multivariate Optimization

In [4]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv) # i.e. optimize(f, x_iv, NelderMead())

 * Status: success

 * Candidate solution
    Minimizer: [1.00e+00, 1.00e+00]
    Minimum:   3.525527e-09

 * Found with
    Algorithm:     Nelder-Mead
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    60
    f(x) calls:    118


* L-BFGS

In [6]:
results = optimize(f, x_iv, LBFGS())
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in "*
"$(results.iterations) iterations")

minimum = 5.378388330692143e-17 with argmin = [0.9999999926662504, 0.9999999853325008] in 24 iterations


As no derivative was given, it used finite differences to approximate the gradient of f(x).

In [7]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv, LBFGS(), autodiff=:forward) # i.e. use ForwardDiff.jl
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in "*
"$(results.iterations) iterations")

minimum = 5.191703158437428e-27 with argmin = [0.999999999999928, 0.9999999999998559] in 24 iterations


Note that we did not need to use `ForwardDiff.jl` directly, as long as our f(x) function was written to be generic (see the generic programming lecture )

Analytical gradient

In [8]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
function g!(G, x)
    G[1] = -2.0 * (1.0 - x[1]) - 400.0 * (x[2] - x[1]^2) * x[1]
    G[2] = 200.0 * (x[2] - x[1]^2)
end

results = optimize(f, g!, x_iv, LBFGS()) # or ConjugateGradient()
println("minimum = $(results.minimum) with argmin = $(results.minimizer) in "*
"$(results.iterations) iterations")

minimum = 5.191703158437428e-27 with argmin = [0.999999999999928, 0.9999999999998559] in 24 iterations


In [9]:
f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2
x_iv = [0.0, 0.0]
results = optimize(f, x_iv, SimulatedAnnealing()) # or ParticleSwarm() or NelderMead()

 * Status: failure (reached maximum number of iterations) (line search failed)

 * Candidate solution
    Minimizer: [1.13e+00, 1.26e+00]
    Minimum:   4.771724e-02

 * Found with
    Algorithm:     Simulated Annealing
    Initial Point: [0.00e+00, 0.00e+00]

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


# Systemps of Equations and Least Squares

# LeastSquaresOptim.jl